## 1️⃣ Environment Setup

Choose one option below:

### Option A: Upload Project Folder (Recommended for small projects)

1. Click the **folder icon** 📁 on the left sidebar
2. Click **Upload** button
3. Upload your entire `llm_dataset` folder (or zip it first)
4. Skip to **Step 2: Verify GPU**

### Option B: Mount Google Drive (Recommended for large projects)

Run the cell below to mount your Google Drive:

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

# Change to your project directory

# Update this path to match your Google Drive structure
%cd /content/drive/MyDrive/llm_dataset

print("✅ Google Drive mounted successfully!")
print(f"📁 Current directory: {!pwd}")

## 2️⃣ Verify GPU is Available

In [ ]:
import torch

print("🖥️  GPU Check:")
print(f"   CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("   ✅ GPU is ready for training!")
else:
    print("   ⚠️  GPU not detected!")
    print("   Go to: Runtime → Change runtime type → Hardware accelerator → GPU")

# Check PyTorch version
print(f"\n📦 PyTorch Version: {torch.__version__}")

## 3️⃣ Install Required Packages

This will install all dependencies (takes ~5 minutes)

In [ ]:
%%capture
# Install dependencies (suppress output to reduce clutter)
!pip install transformers>=4.35.0 datasets>=2.14.0 peft>=0.7.0 accelerate>=0.24.0
!pip install evaluate>=0.4.0 rouge-score>=0.1.2 sentencepiece>=0.1.99 protobuf>=3.20.0

print("✅ All packages installed successfully!")

In [ ]:
# Verify installations
import transformers
import datasets
import peft
import accelerate
import evaluate

print("📦 Package Versions:")
print(f"   transformers: {transformers.__version__}")
print(f"   datasets: {datasets.__version__}")
print(f"   peft: {peft.__version__}")
print(f"   accelerate: {accelerate.__version__}")
print("\n✅ All imports successful!")

## 4️⃣ Verify Dataset Files

In [ ]:
import os
import json

# Check required files
required_files = [
    'model_training/clean_train.jsonl',
    'model_training/clean_val.jsonl',
    'model_training/clean_test.jsonl',
    'scripts/train_model.py'
]

print("📊 Dataset Verification:")
all_present = True

for file in required_files:
    if os.path.exists(file):
        size = os.path.getsize(file) / (1024 * 1024)  # MB
        
        # Count lines for JSONL files
        if file.endswith('.jsonl'):
            with open(file, 'r', encoding='utf-8') as f:
                lines = sum(1 for _ in f)
            print(f"   ✅ {file} ({size:.2f} MB, {lines} examples)")
        else:
            print(f"   ✅ {file} ({size:.2f} MB)")
    else:
        print(f"   ❌ {file} - MISSING")
        all_present = False

if all_present:
    print("\n✅ All required files present!")
    
    # Show sample data
    print("\n📝 Sample Training Data:")
    with open('model_training/clean_train.jsonl', 'r', encoding='utf-8') as f:
        sample = json.loads(f.readline())
        print(f"   Input length: {len(sample['input'])} chars")
        print(f"   Output length: {len(sample['output'])} chars")
        print(f"   Input preview: {sample['input'][:100]}...")
else:
    print("\n❌ Some files are missing. Please upload your project folder.")

## 5️⃣ Start Model Training

This will train FLAN-T5-base with LoRA for 3 epochs (~2-4 hours)

**What happens:**
- Downloads FLAN-T5-base model from Hugging Face
- Applies LoRA adapters (99% memory reduction)
- Trains for 3 epochs with validation
- Saves best checkpoint
- Runs test evaluation

## 🧹 Clear GPU Memory (Run This Before Training!)

**IMPORTANT:** Run this cell to clear any existing GPU memory before starting training.

In [ ]:
import gc
import torch

print("🧹 Clearing GPU memory...")

# Clear Python garbage collector
gc.collect()

# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    print(f"✅ GPU memory cleared!")
    print(f"   Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"   Reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    print(f"   Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")
else:
    print("⚠️  No GPU available")

print("\n💡 Memory optimization tips:")
print("   • If training fails with OOM, restart runtime: Runtime → Restart runtime")
print("   • Close other tabs/programs using GPU")
print("   • The script now uses gradient accumulation (effective batch size = 4)")

## 🚀 Start Training

**What's been optimized for T4 GPU (14.74 GB):**
- ✅ Batch size: 1 (per device)
- ✅ Gradient accumulation: 4 steps (effective batch = 4)
- ✅ Evaluation batch size: 1
- ✅ Max input tokens: 100 (reduced from 128)
- ✅ Max output tokens: 200 (reduced from 256)
- ✅ Memory-efficient evaluation (prediction_loss_only=True)
- ✅ FP16 mixed precision enabled
- ✅ Validation metric: eval_loss (ROUGE computed after training)

**Note:** During training, only validation loss is tracked. Full BLEU/ROUGE evaluation happens after training completes.

This will take ~2-4 hours. Training evaluates at the end of each epoch.

In [ ]:
# Run the training script
!python scripts/train_model.py

### 📊 Monitor Training Progress

While training runs, you'll see:
- **Epoch progress bars** (1/3, 2/3, 3/3)
- **Training loss** (should decrease over time)
- **Validation loss** (eval_loss) at end of each epoch
- **GPU memory usage**

**What you WON'T see during training:**
- ROUGE scores (computed after training in Step 7)
- BLEU scores (computed after training in Step 7)

**Expected Timeline:**
- Epoch 1: ~17-20 minutes training + 1 min validation
- Epoch 2: ~17-20 minutes training + 1 min validation  
- Epoch 3: ~17-20 minutes training + 1 min validation
- Total: ~60-65 minutes (about 1 hour)

## 6️⃣ Check Training Results

In [ ]:
import json

# Check if training completed
if os.path.exists('model_training/fine_tuned_model/test_results.json'):
    print("✅ Training completed successfully!\n")
    
    # Load test results
    with open('model_training/fine_tuned_model/test_results.json', 'r') as f:
        results = json.load(f)
    
    print("📊 Test Set Results:")
    
    # Safely format results (handle both numeric and 'N/A' values)
    eval_loss = results.get('eval_loss', 'N/A')
    if isinstance(eval_loss, (int, float)):
        print(f"   Loss: {eval_loss:.4f}")
    else:
        print(f"   Loss: {eval_loss}")
    
    # Note: ROUGE scores not computed during training (prediction_loss_only=True)
    # They will be computed in Step 7 via evaluate_model.py
    print("\n💡 Note: ROUGE/BLEU scores not computed during training.")
    print("   Run Step 7 (evaluate_model.py) for comprehensive metrics.")
    
    # Check model files
    print("\n📁 Model Files:")
    model_dir = 'model_training/fine_tuned_model'
    if os.path.exists(model_dir):
        for file in os.listdir(model_dir):
            file_path = os.path.join(model_dir, file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path) / (1024 * 1024)
                print(f"   {file} ({size:.2f} MB)")
    else:
        print("   Model directory not found")
else:
    print("⏳ Training not yet complete or failed.")
    print("   Check the output above for errors.")

## 7️⃣ Run Comprehensive Evaluation

In [ ]:
# Run evaluation script
!python scripts/evaluate_model.py \
  --model_path model_training/fine_tuned_model \
  --test_data model_training/clean_test.jsonl \
  --output model_training/evaluation_report.csv

In [ ]:
# View evaluation summary
import pandas as pd

if os.path.exists('model_training/evaluation_report.csv'):
    df = pd.read_csv('model_training/evaluation_report.csv')
    
    print("📊 Evaluation Summary:")
    print(f"   Total examples: {len(df)}")
    
    # Show available columns
    print(f"\n📋 Available columns: {', '.join(df.columns.tolist())}")
    
    # Try to display metrics with error handling
    print("\n📈 Metrics:")
    
    # Check and display BLEU scores
    bleu_cols = [col for col in df.columns if col.startswith('bleu')]
    if bleu_cols:
        for col in bleu_cols:
            print(f"   Average {col.upper()}: {df[col].mean():.4f}")
    else:
        print("   ⚠️  BLEU scores not found in report")
    
    # Check and display ROUGE scores
    rouge_cols = [col for col in df.columns if 'rouge' in col.lower()]
    if rouge_cols:
        for col in rouge_cols:
            print(f"   Average {col}: {df[col].mean():.4f}")
    else:
        print("   ⚠️  ROUGE scores not found in report")
    
    # Check for metadata consistency
    if 'metadata_consistency' in df.columns:
        print(f"   Metadata Consistency: {df['metadata_consistency'].mean():.2%}")
    
    # Show top 5 if we have rouge_l column
    if 'rouge_l' in df.columns:
        print("\n🌟 Top 5 Best Predictions (by ROUGE-L):")
        display_cols = ['rouge_l'] + [c for c in ['bleu_4', 'bleu_1'] if c in df.columns]
        top5 = df.nlargest(5, 'rouge_l')[display_cols]
        print(top5.to_string(index=False))
        
        # Show distribution
        print("\n📈 ROUGE-L Distribution:")
        print(df['rouge_l'].describe())
    
    # Show first few rows for inspection
    print("\n📄 Sample of evaluation data:")
    print(df.head(3).to_string())
else:
    print("❌ Evaluation report not found.")
    print("   Make sure to run the previous cell to generate the evaluation report.")

## 8️⃣ Test Inference Pipeline

In [ ]:
# Test with example file
!python scripts/claim_report_generation.py \
  --model_path model_training/fine_tuned_model \
  --input model_training/example_input.json

In [ ]:
# Interactive test - Load model and generate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

print("🚀 Loading model for interactive testing...")

# Load model and tokenizer
model_path = 'model_training/fine_tuned_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

if torch.cuda.is_available():
    model = model.to('cuda')
    print("✅ Model loaded on GPU\n")
else:
    print("✅ Model loaded on CPU\n")

# Test samples
test_samples = [
    {
        "shipment_id": "SH12345",
        "product_id": "PRD789",
        "damage_type": "dent",
        "damage_severity": "moderate",
        "damage_location": "corner",
        "carrier": "FastShip Logistics",
        "inspection_date": "2025-11-27"
    },
    {
        "shipment_id": "SH67890",
        "product_id": "PRD456",
        "damage_type": "wet",
        "damage_severity": "severe",
        "damage_location": "entire package",
        "carrier": "QuickMove Express",
        "inspection_date": "2025-11-27"
    }
]

# Generate reports
for i, sample in enumerate(test_samples, 1):
    print(f"\n{'='*80}")
    print(f"Test Case {i}: {sample['damage_type'].upper()} damage")
    print(f"{'='*80}")
    
    # Create input text
    input_text = json.dumps(sample)
    
    # Tokenize
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    if torch.cuda.is_available():
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=512,
            num_beams=4,
            temperature=0.7,
            do_sample=False
        )
    
    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"\n📝 Generated Claim Report:\n")
    print(generated_text)

print(f"\n{'='*80}")
print("✅ Inference testing complete!")
print(f"{'='*80}")

## 9️⃣ Download Results

Download trained model and results to your computer

In [ ]:
# Create a zip file of results
import shutil

print("📦 Creating zip file of results...")

# Create zip of fine_tuned_model
if os.path.exists('model_training/fine_tuned_model'):
    shutil.make_archive('fine_tuned_model', 'zip', 'model_training/fine_tuned_model')
    print("   ✅ fine_tuned_model.zip created")

# Create zip of evaluation results
files_to_zip = [
    'model_training/evaluation_report.csv',
    'model_training/evaluation_samples.txt',
    'model_training/fine_tuned_model/test_results.json'
]

print("\n📥 Files ready for download:")
print("   1. fine_tuned_model.zip (trained model)")
print("   2. evaluation_report.csv (metrics)")
print("   3. evaluation_samples.txt (sample predictions)")
print("\n💡 Use the Files panel (📁) on the left to download these files.")

In [ ]:
# Optional: Copy results to Google Drive
if os.path.exists('/content/drive'):
    print("💾 Copying results to Google Drive...")
    
    # Create output directory in Drive
    drive_output = '/content/drive/MyDrive/Week5_Results'
    os.makedirs(drive_output, exist_ok=True)
    
    # Copy key files
    if os.path.exists('fine_tuned_model.zip'):
        shutil.copy('fine_tuned_model.zip', drive_output)
        print("   ✅ Model copied to Drive")
    
    if os.path.exists('model_training/evaluation_report.csv'):
        shutil.copy('model_training/evaluation_report.csv', drive_output)
        print("   ✅ Evaluation report copied to Drive")
    
    print(f"\n✅ Results saved to: {drive_output}")
else:
    print("ℹ️  Google Drive not mounted. Files available in Colab session only.")

## 🎯 Summary

### Completed Tasks:
- ✅ Environment setup (GPU verified)
- ✅ Dependencies installed
- ✅ Model training (FLAN-T5-base with LoRA)
- ✅ Comprehensive evaluation
- ✅ Inference testing
- ✅ Results ready for download

### Next Steps:
1. **Download Results:** Use Files panel to download `fine_tuned_model.zip` and `evaluation_report.csv`
2. **Review Metrics:** Check ROUGE-L and BLEU scores in evaluation report
3. **Test Locally:** Use the trained model with `claim_report_generation.py`
4. **Deploy API (Optional):** Run `app.py` for FastAPI deployment
5. **Create Demo Video:** Record inference examples

### Key Deliverables:
- ✅ `fine_tuned_model/` - Trained model weights
- ✅ `evaluation_report.csv` - Detailed metrics
- ✅ `claim_report_generation.py` - Inference pipeline
- ✅ Test results and sample predictions

---

**🎉 Week 5 Training Complete!**

**Training Time:** Check the output above for actual duration

**Quality:** Review ROUGE-L score (target: >0.50)

**Status:** Ready for deployment and demo video creation

---